# Лабораторная работа №5: Методы, основанные на деревьях решений. Регрессионные деревья. Деревья классификации. Случайный лес. Бустинг.
В практических примерах ниже показано:
+ как делать перекодировку признаков в номинальной и порядковой шкалах
+ как вырастить дерево и сделать обрезку его ветвей
+ как настроить модель бэггинга
+ как вырастить случайный лес
+ как настроить модель бустинга на деревьях решений
+ как подбирать настроенные параметры моделей методом сеточного поиска
Точность всех моделей оценивается методом перекрёстной проверки по 5 блокам.

# Загружаем пакеты

In [1]:
import sklearn
print(sklearn.__version__)

0.20.0


In [2]:
# загрузка пакетов: инструменты --------------------------------------------
# работа с массивами
import numpy as np
# фреймы данных
import pandas as pd
# графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# загрузка файлов по URL
import urllib
# проверка существования файла на диске
from pathlib import Path
# для форматирования результатов с помощью Markdown
from IPython.display import Markdown, display
# перекодировка категориальных переменных
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# хи-квадрат тест на независимость по таблице сопряжённости
from scipy.stats import chi2_contingency
# для таймера
import time
# загрузка пакетов: данные -------------------------------------------------
from sklearn import datasets
# загрузка пакетов: модели -------------------------------------------------
# дерево классификации
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# бэггинг
from sklearn.ensemble import BaggingClassifier
# случайный лес
from sklearn.ensemble import RandomForestClassifier
# бустинг
from sklearn.ensemble import GradientBoostingClassifier
# сводка по точности классификации
from sklearn.metrics import classification_report

ImportError: cannot import name 'plot_tree'

In [ ]:
# константы
# ядро для генератора случайных чисел
my_seed = 18
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
# примеры стилей и шаблонов графиков:
# http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру

In [ ]:
# функция форматирования результатов с использованием Markdown
def printmd(string):
 display(Markdown(string))

# функции для попарной конкатенации элементов двух списков
concat_func_md = lambda x, y: '`' + str(x) + "`:&ensp;&ensp;&ensp;&ensp;" + str(y)
concat_func = lambda x, y: str(x) + ' ' * 4 + str(y)
# функция, которая строит график важности признаков в модели случайного леса
# источник: https://www.analyseup.com/learn-python-for-data-science/python-rand
def plot_feature_importance(importance, names, model_type):
 #Create arrays from feature importance and feature names
 feature_importance = np.array(importance)
 feature_names = np.array(names)
 #Create a DataFrame using a Dictionary
 data={'feature_names':feature_names,'feature_importance':feature_importance}
 fi_df = pd.DataFrame(data)
 #Sort the DataFrame in order decreasing feature importance
 fi_df.sort_values(by=['feature_importance'], ascending=False,
 inplace=True)
 #Define size of bar plot
 plt.figure(figsize=(10,8))
 #Plot Searborn bar chart
 sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
 #Add chart labels
 plt.title('Важность признаков в модели: ' + model_type)
 plt.xlabel('Важность признака')
 plt.ylabel('')

# Загружаем данные:

In [ ]:
fileURL = "https://raw.githubusercontent.com/ania607/ML/main/data/default_of_credit_card_clients.csv"
DF_all = pd.read_csv(fileURL)
# загружаем таблицу и превращаем её во фрейм
# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_all.shape)

### Откладываем 15% наблюдений для прогноза
+ оставляем только нужные перменные

In [ ]:
DF_all.drop(['PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT4', 'BILL_AMT5',
             'BILL_AMT6', 'PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4','PAY_AMT5',
             'PAY_AMT6'], axis=1, inplace=True)
# наблюдения для моделирования
DF = DF_all.sample(frac = 0.85, random_state = my_seed)
# отложенные наблюдения
DF_predict = DF_all.drop(DF.index)
# первые 5 строк фрейма у первых 7 столбцов
DF.iloc[:, :7].head(5)

In [ ]:
# первые 5 строк фрейма у столбцов 8-11
DF.iloc[:, 7:11].head(5)

In [ ]:
# первые 5 строк фрейма у столбцов 12-14
DF.iloc[:, 11:].head(5)

In [ ]:
# типы столбцов фрейма
DF.dtypes

Проверим, нет ли в таблице пропусков.

In [ ]:
# считаем пропуски в каждом столбце
DF.isna().sum()

Пропусков не обнаружено.

## Предварительный анализ данных
### Описательные статистики
Считаем доли классов целевой переменной ***Y*** .

In [ ]:
# метки классов
DF.Y.unique()

In [ ]:
# доли классов
np.around(DF.Y.value_counts() / len(DF.index), 3)

Итак, всего целевых классов два, и их доли сильно разняться, с перевесом в пользу класса
'0'. Все объясняющие переменные набора данных непрерывные. Рассчитаем для них
описательные статистики.

In [ ]:
# описательные статистики
DF.iloc[:, :6].describe()

In [ ]:
# описательные статистики
DF.iloc[:, 6:11].describe()

Выводы по описательным статистикам: значения объясняющих переменных положительные кроме 'PAY_2', 'PAY_3'
масштабы измерения отличаются. Для работы с методами снижения размерности и
регуляризации понадобится стандартизация значений.

## Визуализация разброса переменных внутри классов
Поскольку в наборе данных 11 объясняющих переменных, и все они непрерывные (кроме поля 'SEX'), анализ
матричного графика разброса будет затруднительным. Построим коробчатые диаграммы для
объясняющих переменных, чтобы сравнить средние уровни и разброс по классам.

In [ ]:
# создаём полотно и делим его на четыре части
fig = plt.figure(figsize=(12, 5))
gs = mpl.gridspec.GridSpec(1, 5)
ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])
ax3 = plt.subplot(gs[0, 2])
ax4 = plt.subplot(gs[0, 3])
ax5 = plt.subplot(gs[0, 4])
axs = [ax1, ax2, ax3, ax4, ax5]
cols_loop = list(DF.columns[:5].values)
for col_name in cols_loop :
 i = cols_loop.index(col_name)
 sns.boxplot(x='Y', y=col_name, data=DF, ax=axs[i])
 axs[i].set_ylabel(col_name)
 axs[i].set_title(col_name)
    
# корректируем расположение графиков на полотне
gs.tight_layout(plt.gcf())
plt.show()

In [ ]:
# создаём полотно и делим его на четыре части
fig = plt.figure(figsize=(12, 5))
gs = mpl.gridspec.GridSpec(1, 5)
ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])
ax3 = plt.subplot(gs[0, 2])
ax4 = plt.subplot(gs[0, 3])
ax5 = plt.subplot(gs[0, 4])
axs = [ax1, ax2, ax3, ax4, ax5]
cols_loop = list(DF.columns[5:10].values)
for col_name in cols_loop :
 i = cols_loop.index(col_name)
 sns.boxplot(x='Y', y=col_name, data=DF, ax=axs[i])
 axs[i].set_ylabel(col_name)
 axs[i].set_title(col_name)
# корректируем расположение графиков на полотне
gs.tight_layout(plt.gcf())
plt.show()

In [ ]:
# создаём полотно и делим его на четыре части
fig = plt.figure(figsize=(7.2, 5))
gs = mpl.gridspec.GridSpec(1, 3)
ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])
ax3 = plt.subplot(gs[0, 2])
axs = [ax1, ax2, ax3]
cols_loop = list(DF.columns[10:11].values)
for col_name in cols_loop :
 i = cols_loop.index(col_name)
 sns.boxplot(x='Y', y=col_name, data=DF, ax=axs[i])
 axs[i].set_ylabel(col_name)
 axs[i].set_title(col_name)
# корректируем расположение графиков на полотне
gs.tight_layout(plt.gcf())
plt.show()


На графиках  сходство в медианах и разбросе между классами прослеживается практически по
всем объясняющим переменным кроме `LIMIT_BAL, PAY_0, PAY_2, PAY_3`. Меньше всего различаются коробчатые диаграммы по
переменной `EDUCATION, MARRIAGE, AGE, BILL_AMT1, BILL_AMT2, BILL_AMT3` . Это говорит о том, классы по зависимой переменной **Y** неплохо
разделяются по всем объясняющим переменным.

## Корреляционный анализ
Теперь посмотрим на взаимодействие объясняющих переменных.

In [ ]:
# рассчитываем корреляционную матрицу
corr_mat = DF.drop('Y', axis=1).corr()
col_names = DF.drop('Y', axis=1).columns
# переключаем стиль оформления, чтобы убрать сетку с тепловой карты
mpl.style.use('seaborn-white')
# рисуем корреляционную матрицу
f = plt.figure(figsize=(10, 8))
plt.matshow(corr_mat, fignum=f.number, cmap='PiYG')
# координаты для названий строк и столбцов
tics_coords = np.arange(0, len(col_names))
# рисуем подписи
plt.xticks(tics_coords, col_names, fontsize=14, rotation=90)
plt.yticks(tics_coords, col_names, fontsize=14)
# настраиваем легенду справа от тепловой карты
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
cb.ax.tick_params(labelsize=14)
plt.show()

Между объясняющими переменными обнаруживаются как прямые, так и обратные линейные
взаимосвязи. Выведем все значимые коэффициенты в одной таблице и определим
минимальный / максимальный из них.

## Модель дерева
В этом разделе построим: 
+ дерево классификации
+ дерево классификации с обрезкой ветвей

## Дерево на всех признаках
Построим модель и выведем изображение дерева в виде текста.

In [ ]:
# выращиваем дерево на всех объясняющих
X = DF.drop(['Y'], axis=1)
y = DF['Y']
# классификатор
cls_one_tree = DecisionTreeClassifier(criterion='entropy',
 random_state=my_seed)
tree_full = cls_one_tree.fit(X, y)
# выводим количество листьев (количество узлов)
tree_full.get_n_leaves()

In [ ]:
# глубина дерева: количество узлов от корня до листа
# в самой длинной ветви
tree_full.get_depth()

Очевидно, дерево получилось слишком большое для отображения в текстовом формате.
Графическая визуализация тоже не поможет в данном случае. Посчитаем показатели точности
с перекрёстной проверкой.

In [ ]:
# будем сохранять точность моделей в один массив:
score = list()
score_models = list()
# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator=cls_one_tree, X=X, y=y, cv=5,
 scoring='accuracy')
# записываем точность
score.append(np.around(np.mean(cv), 3))
score_models.append('one_tree')
print('Acc с перекрёстной проверкой',
 '\nдля модели', score_models[0], ':', score[0])


## Дерево с обрезкой ветвей
Подберём оптимальное количество ветвей, которое максимизирует , для экономии
времени рассчитанный методом проверочной выборки.

In [ ]:
# рассчитываем параметры alpha для эффективных вариантов обрезки ветвей
path = cls_one_tree.cost_complexity_pruning_path(X, y)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print('Всего значений alpha:', len(ccp_alphas))
print('Энтропия листьев для первых 5 значений alpha:', impurities[:5])

In [ ]:
# изображаем на графике
plt.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
plt.xlabel("значение гиперпараметра alpha")
plt.ylabel("общая энтропия листьев дерева")
plt.title("Изменение показателя нечистоты узлов с ростом alpha")
plt.show()


In [ ]:
# обучающая и тестовая выборки, чтобы сэкономить время
X_train, X_test, y_train, y_test = train_test_split(X, y,
 random_state=my_seed)
# модели
clfs = list()
# таймер
tic = time.perf_counter()
# цикл по значениям alpha
for ccp_alpha in ccp_alphas:
 clf = DecisionTreeClassifier(random_state=my_seed, ccp_alpha=ccp_alpha)
 clf.fit(X_train, y_train)
 clfs.append(clf)
# таймер
toc = time.perf_counter()
print(f"Расчёты по обрезке дерева заняли {toc - tic:0.2f} секунд")

In [ ]:
# извлекаем характеристики глубины и точности
# таймер
tic = time.perf_counter()
node_counts = [clf.tree_.node_count for clf in clfs]
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]
# таймер
toc = time.perf_counter()
print(f"Расчёты показателей точности заняли {toc - tic:0.2f} секунд")

In [ ]:

# изображаем на графике
fig, ax = plt.subplots(1, 2)
# график глубины дерева
ax[0].plot(ccp_alphas, node_counts, marker=',', drawstyle="steps-post")
ax[0].set_xlabel("значение гиперпараметра alpha")
ax[0].set_ylabel("количество узлов")
ax[0].set_title("Сложность модели vs alpha")
# график точности
ax[1].plot(ccp_alphas, train_scores, marker=',', label='train',
 drawstyle="steps-post")
ax[1].plot(ccp_alphas, test_scores, marker=',', label='test',
 drawstyle="steps-post")
ax[1].set_xlabel("значение гиперпараметра alpha")
ax[1].set_ylabel("Acc")
ax[1].set_title("Точность модели vs alpha")
fig.tight_layout()


Находим оптимальный размер дерева по максимуму на тестовой выборке.

In [ ]:
# оптимальное количество узлов
opt_nodes_num = node_counts[test_scores.index(max(test_scores))]
# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator=clfs[opt_nodes_num], X=X, y=y, cv=5,
 scoring='accuracy')
# записываем точность
score.append(np.around(np.mean(cv), 3))
score_models.append('pruned_tree')
print('Оптимальное количество узлов:', opt_nodes_num,
 '\nсоответствующая Acc на тестовой:', np.around(max(test_scores), 3),
 '\n\nAcc с перекрёстной проверкой',
 '\nдля модели', score_models[1], ':', score[1])

Посмотрим на характеристики глубины и сложности построенного дерева с обрезкой ветвей.

In [ ]:
# выводим количество листьев (количество узлов)
clfs[opt_nodes_num].get_n_leaves()

In [ ]:
# глубина дерева: количество узлов от корня до листа
# в самой длинной ветви
clfs[opt_nodes_num].get_depth()


## Бустинг
Подберём сеточным поиском настроечные параметры модели:
+ B число деревьев
+ λ – скорость обучения
+ d – глубина взаимодействия предикторов

In [ ]:
# разбиения для перекрёстной проверки
kfold = KFold(n_splits=5, random_state=my_seed, shuffle=True)
# обучаем модель с параметрами по умолчанию
clf_tst = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
 max_depth=1, random_state=my_seed)
cv = cross_val_score(clf_tst, X, y, cv=kfold, scoring='accuracy')
np.around(np.mean(cv), 3)

In [ ]:
# настроим параметры бустинга с помощью сеточного поиска
param_grid = {'n_estimators' : [10, 20, 30, 40, 50],
 'learning_rate' : np.linspace(start=0.01, stop=0.25, num=15),
 'max_depth' : [1, 2]}
# таймер
tic = time.perf_counter()
clf = GridSearchCV(GradientBoostingClassifier(),
 param_grid, scoring='accuracy', cv=kfold)
boost_tree = clf.fit(X, y)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

In [ ]:
# точность лучшей модели
np.around(boost_tree.best_score_, 3)


In [ ]:
# параметры лучшей модели
print('n_estimators:',
 boost_tree.best_estimator_.get_params()['n_estimators'],
 '\nlearning_rate:',
 boost_tree.best_estimator_.get_params()['learning_rate'],
 '\nmax_depth:',
 boost_tree.best_estimator_.get_params()['max_depth'])


In [ ]:
# записываем точность
score.append(np.around(boost_tree.best_score_, 3))
score_models.append('boost_tree_GS')
print('Acc с перекрёстной проверкой',
 '\nдля модели', score_models[3], ':', score[3])

## Прогноз на отложенные наблюдения по лучшей модели
Ещё раз посмотрим на точность построенных моделей.

In [ ]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

Все модели показывают среднюю точность по показателю , при этом самой точной
оказывается модель случайного леса. Сделаем прогноз на отложенные наблюдения.

In [ ]:
# данные для прогноза
X_pred = DF_predict.drop(['Y'], axis=1)
# строим прогноз
y_hat = random_forest.best_estimator_.predict(X_pred)
# характеристики точности
print(classification_report(DF_predict_num['Y'], y_hat))